In [1]:
import pymongo
import requests
import json
import time
from datetime import datetime
from threading import Thread

client = pymongo.MongoClient("localhost:27017")
db = client.MasterThesis
collection = db["CoronaVirus"]
consumer_key,consumer_secret = open("keys.txt","r").readlines()

In [ ]:
def get_bearer_token(key, secret):
    response = requests.post(      
        "https://api.twitter.com/oauth2/token",
        auth=(key, secret),
        data={'grant_type': 'client_credentials'},
        headers={"User-Agent": "TwitterDevCovidmmazzola"})

    if response.status_code is not 200:
        raise Exception(f"Cannot get a Bearer token (HTTP %d): %s" % (response.status_code, response.text))
    try:
        body = response.json()
        return body['access_token']
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
    except requests.exceptions.SSLError:
        r.status_code = "Vaffanculoooooo"
    
def save_data_mongo(item):
    collection.insert_one(item)
    
def stream_connect(partition):
    response = requests.get("https://api.twitter.com/labs/1/tweets/stream/covid19?partition={}".format(partition),
                            headers={"User-Agent": "TwitterDevCovidmmazzola",
                                     "Authorization": "Bearer {}".format(get_bearer_token(consumer_key, consumer_secret))},
                            stream=True,verify=False)
    for response_line in response.iter_lines():
        if response_line:
            save_data_mongo(json.loads(response_line))

def main():
    timeout = 0
    while True:
        for partition in range(1, 5):
            Thread(target=stream_connect, args=(partition,)).start()
        time.sleep(2 ** timeout * 1000)
        timeout += 1

if __name__ == "__main__":
    main()
    

Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\mmazzola\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py", line 456, in wrap_socket
    cnx.do_handshake()
  File "C:\Users\mmazzola\AppData\Local\Continuum\anaconda3\lib\site-packages\OpenSSL\SSL.py", line 1915, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "C:\Users\mmazzola\AppData\Local\Continuum\anaconda3\lib\site-packages\OpenSSL\SSL.py", line 1639, in _raise_ssl_error
    raise SysCallError(errno, errorcode.get(errno))
OpenSSL.SSL.SysCallError: (10054, 'WSAECONNRESET')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\mmazzola\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "C:\Users\mmazzola\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 343, in _make_requ